# Daily Reservation Report

In [1]:
#connect to database
import pyodbc 

#build report
import pandas as pd
import numpy as np

# create and send email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.utils import formatdate
from email import encoders
import smtplib



In [2]:
# connect to database

try: 
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=SERVER_NAME;'                
                      'Database=reportingdb;'
                      'Trusted_Connection=yes;')
    print("Successfully connected to the database")
except: print("Did not connect to  the database")

Successfully connected to the database


In [3]:
# Load into Pandas

# --  from departRpt table: lastName, resNum, puDate, puLoc, vehClass, retDate, retLoc, bookingLoc, dateBooked
# -- from wwr_resdic table: emplNum, 
# --,departRpt.tourOp,   departRpt.cancelDate,   departRpt.raNum

cursor = conn.cursor()
query = """


SELECT        departRpt.lastName AS CUSTOMER, departRpt.resNum AS "RESERVATION #" ,
              departRpt.puDate AS "PICK-UP DATE", departRpt.puLoc AS "PICK-UP LOCATION",
			  departRpt.vehClass AS "VEHICLE CLASS", departRpt.retDate AS "RETURN DATE",
			  departRpt.retLoc AS "RETURN LOCATION", departRpt.bookingLoc AS "BOOKING LOCATION",
			  departRpt.dateBooked AS "DATE BOOKED" ,wwr_resdisc.emplNum AS "BOOKED BY" 
			  

FROM            departRpt LEFT OUTER JOIN
                         wwr_resdisc ON departRpt.resNum = wwr_resdisc.resNum
WHERE        departRpt.dateBooked = CAST(GETDATE() - 1 AS DATE)

"""
# results = cursor.execute(query).fetchall()

df=pd.read_sql_query(query,conn)


df.to_excel('Daily_Reservation_Report.xlsx', index = False)
conn.close()

c:\Users\mriedeman\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
# view table 
df.tail()

,CUSTOMER,RESERVATION #,PICK-UP DATE,PICK-UP LOCATION,VEHICLE CLASS,RETURN DATE,RETURN LOCATION,BOOKING LOCATION,DATE BOOKED,BOOKED BY
130,ROZENBOOM,503773,2022-06-10,DEN,C25,2022-06-19,DEN,CEN,2022-05-22,WEB
131,RAGHAVENDRA,503791,2022-09-30,712,C25,2022-10-03,712,CEN,2022-05-22,CAR09
132,BASCOM,503805,2022-07-22,SLC,C25,2022-07-31,SLC,CEN,2022-05-22,WEB
133,WOLF,L72299,2022-06-20,SFO,C21,2022-06-29,SFO,INT,2022-05-22,IBEXML
134,MCMILLAN,L72310,2023-07-29,323,C25,2023-08-05,323,INT,2022-05-22,TRAILF


In [11]:
# send email

#create multipart email object
email = MIMEMultipart()

sender = 'MY_OUTLOOK_EMAIL@COMPANY_NAME.COM'
recipients = ['EXAMPLE_EMAIL@GMAIL.COM', 'EXAMPLE_EMAIL@OUTLOOK.COM']

email_txt= "**Automated Email**\n\nGentlemen,\n\nHere is the daily reservation report.\n\nMichael"

email['From'] = sender
email['To'] = ", ".join(recipients)
email['Subject'] = "Daily Reservation Report"

email.attach(MIMEText(email_txt))

# https://kb.iu.edu/d/agtj Explain "application/octet_stream"
# A MIME attachment with the content type "application/octet-stream" is a binary file. 
# Typically, it will be an application or a document that must be opened in an application, such as a spreadsheet or word processor.
mimeattachment = MIMEBase('application', "octet-stream")

#Set the entire message object’s payload to payload. It is the client’s responsibility to ensure the payload invariants. Optional charset sets the message’s default character set; see set_charset() for details.

# This is a legacy method. On the EmailMessage class its functionality is replaced by set_content().
mimeattachment.set_payload(open("Daily_Reservation_Report.xlsx", "rb").read())


# Encode file in ASCII characters to send by email
encoders.encode_base64(mimeattachment)


mimeattachment.add_header('Content-Disposition', 'attachment; filename="Daily_Reservation_Report.xlsx"')
email.attach(mimeattachment)
#initialise connection to outlook mail server
server = smtplib.SMTP('smtp-mail.outlook.com', '587')

#hostname to identify itself
server.ehlo()  

#puts the connection to the SMTP server into TLS mode.

#TLS :

# Transport Layer Security (TLS) encrypts data sent over the Internet to ensure that 
# eavesdroppers and hackers are unable to see what you transmit which is particularly
#  useful for private and sensitive information such as passwords, credit card numbers,
#  and personal correspondence.

server.starttls()

#Log in on an SMTP server that requires authentication.
server.login('MY_OUTLOOK_EMAIL@COMPANY_NAME.COM', 'PASSWORD')

text = email.as_string()

#Compile email: From, To, Email body
server.sendmail(sender, recipients, text)
  
    
server.quit()

Content-Type: multipart/mixed; boundary="===============6654255316112981930=="
MIME-Version: 1.0
From: mriedeman@cruiseamerica.com
To: mriedeman@cruiseamerica.com, kkirkpatrick@cruiseamerica.com
Subject: Daily Reservation Report

--===============6654255316112981930==
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

**Automated Email**

Gentlemen,

Here is the daily reservation report.

Michael
--===============6654255316112981930==
Content-Type: application/octet-stream
MIME-Version: 1.0
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="Daily_Reservation_Report.xlsx"

UEsDBBQAAAAIAAAAPwD4SqchUQEAAJAEAAATAAAAW0NvbnRlbnRfVHlwZXNdLnhtbK2Uy07DMBBF
90j8Q+QtStyyQAg17YLHEipRPsDYk8aqY1ueaWn/nkn6ECBoi9pNrGTu3HPHjjwYLRuXLSChDb4U
/aInMvA6GOunpXibPOW3IkNS3igXPJRiBShGw8uLwWQVATPu9liKmijeSYm6hkZhESJ4rlQhNYr4
NU1lVHqmpiCve70bqYMn8JRT6yGGgweo1NxR9rjkz+skCRyK7H4tbFmlUDE6qxVxXS68+UHJN4SC
OzsN1jbiFQuE/JXQVv4GbPpeeGuSNZCNVaJn1bBKmq

(221, b'2.0.0 Service closing transmission channel')